Decision Tree features

In [ ]:
X = df_clientes_csv_rfm_with_label.drop(columns=["class_desc_short","FECHA_REGISTRO","FECHA_CLIENTE","DESC_SECTOR"])
y = df_clientes_csv_rfm_with_label["class_desc_short"].to_numpy()

Oversampling

In [ ]:
from imblearn.over_sampling import RandomOverSampler
X_ros, y_ros = RandomOverSampler(random_state=10).fit_resample(df_rfm,cluster_labels)

Plot 3d Cluster

In [109]:
# Función para visualizar los grupos RFM en 3D. Puede recibir las etiquetas de los datos
def plot_rfm_3d_cluster(df, labels):
    fig = plt.figure(figsize=(6, 4))
    ax1 = fig.add_subplot(111, projection='3d')
    scatter = ax1.scatter(xs=df["frequency"], ys=df["monetary"], zs=df["recency"], c=labels, marker='o')

    # Add labels
    ax1.set_xlabel('frequency')
    ax1.set_ylabel('monetary')
    ax1.set_zlabel('recency')

In [ ]:
# Código original extraido de CHATGPT y ajustado posteriormente

from sklearn.metrics import silhouette_score, silhouette_samples

# Fit k-means with different cluster sizes
range_n_clusters = [2, 3, 4, 5, 6, 7, 8]
for n_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(df_rfm_scaled)
    
    # Calculate the average silhouette score
    silhouette_avg = silhouette_score(df_rfm_scaled, cluster_labels)
    print(f"For n_clusters = {n_clusters}, the average silhouette score is: {silhouette_avg:.3f}")

    # Calculate the silhouette score for each sample
    sample_silhouette_values = silhouette_samples(df_rfm_scaled, cluster_labels)

    # Plot silhouette analysis
    fig, ax1= plt.subplots(1, 1, figsize=(10, 3))
    y_lower = 10
    for i in range(n_clusters):
        # Aggregate silhouette scores for samples in the current cluster
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]
        ith_cluster_silhouette_values.sort()
        
        # Calculate where the cluster silhouette bar ends
        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i
        
        # Plot silhouette values
        color = plt.cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                         0, ith_cluster_silhouette_values,
                         facecolor=color, edgecolor=color, alpha=0.7)
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
        
        y_lower = y_upper + 10  # Adjust for the next cluster

        ax1.set_title(f"Silhouette plot for {n_clusters} clusters")
        ax1.set_xlabel("Silhouette coefficient values")
        ax1.set_ylabel("Cluster")
        ax1.axvline(x=silhouette_avg, color="red", linestyle="--")
        
    plt.xticks(ticks=np.arange(0, 1, 0.05));
    plt.grid(axis="x")
    plt.show()


### Cluster Jerarquico

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage

Vamos a generar un dendograma con Agglomerative Clustering, y con el enlace "single", que es mejor para grupos no circulares.

In [ ]:
# Generate linkage matrix for dendrogram
Z = linkage(df_rfm_scaled, method='single')

# Plot the dendrogram
plt.figure(figsize=(10, 3))
dendrogram(Z, truncate_mode="level", p=4)
plt.title('Dendrogram - Agglomerative Clustering')
plt.xlabel('Data Points')
plt.ylabel('Euclidean Distance')
plt.grid(True)
plt.show()

Observando el Dendograma, 

In [ ]:
agg_labels = AgglomerativeClustering(n_clusters=5, linkage='single').fit_predict(df_rfm_scaled)

In [ ]:
plot_rfm_3d_cluster(df_rfm, agg_labels)